Spark-Session erstellen und notwendige Frameworks importieren

In [2]:
import findspark
findspark.init()
import pyspark

conf = pyspark.SparkConf()
conf.set("spark.driver.bindAddress", "127.0.0.1")  # Localhost
conf = conf.setMaster("local[*]")
conf = conf.setAppName("App")


sc = pyspark.SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/19 12:34:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Analyse Funktionen für verschiedene Eigenschaften

# Datenverteilung im CLuster
def analyze_partition_distribution(rdd):
    """
    Analysiert die Verteilung der Daten in einem RDD über die Partitionen.
    
    :param rdd: Das RDD, das analysiert werden soll.
    """
    # Funktion, um die Größe jeder Partition zu berechnen
    def partition_sizes(index, iterator):
        yield index, sum(1 for _ in iterator)

    # Daten je Partition sammeln
    partition_info = rdd.mapPartitionsWithIndex(partition_sizes).collect()
    
    # Ergebnisse ausgeben
    print("\n=== Verteilung der Daten auf Partitionen ===")
    print(f"Anzahl der Datensätze: {rdd.count()}")
    print("Datenverteilung auf Partitionen:")
    for partition, size in partition_info:
        print(f"Partition {partition}: {size} Datensätze")


# Fehlertoleranz

Einlesen der Datei mit allen weltweiten Flugplätzen und Filtern auf die Region "EU" sowie Airport Typ "medium_airport" und "large_airport"
(Hier stimmt was noch nicht, laut CSV müssten es eigentlich mehr als 976 Flugplätze sein!)

In [ ]:
airportRDD = sc.textFile("data/airports.csv")
header = airportRDD.first()

airportRDD = (
    airportRDD.filter(lambda line: line != header)
    .filter(lambda line: line.split(",")[7].strip('"') in ["EU"])
    .filter(lambda line: line.split(",")[2].strip('"') in ["medium_airport", "large_airport"])
)

Anzahl_Flughäfen_gesamt = airportRDD.count()

Anzahl_Flugplätze_kategorisiert = (
    airportRDD.map(lambda line: line.split(",")[2].strip('"'))
    .countByValue()
)

print(f"Die Anzahl der europäischen mittelgroßen und großen Flughäfen beträgt: {Anzahl_Flughäfen_gesamt}")
print(f"Davon mittelgroße Flughäfen: {Anzahl_Flugplätze_kategorisiert.get("medium_airport", 0)}")
print(f"Davon große Flughäfen: {Anzahl_Flugplätze_kategorisiert.get("large_airport", 0)}")


# Untersuchung der Datenverarbeitung
analyze_partition_distribution(airportRDD)

Die Anzahl der europäischen mittelgroßen und großen Flughäfen beträgt: 976
Davon mittelgroße Flughäfen: 864
Davon große Flughäfen: 112

=== Verteilung der Daten auf Partitionen ===
Anzahl der Datensätze: 976
Datenverteilung auf Partitionen:
Partition 0: 374 Datensätze
Partition 1: 602 Datensätze


Einlesen und filtern aller weltweit zugelassenen Flugobjekte, sortiert nach den Airlines mit den meisten Flugzeugen

In [18]:
aircraft_input_rdd= sc.textFile('data/aircraft-database-complete-2024-10.csv') 
header = aircraft_input_rdd.first()

aircraft_raw_rdd = aircraft_input_rdd.filter(lambda line: line != header)

Anzahl_registrierte_Flugobjekte = aircraft_raw_rdd.count()
print(f"Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte {Anzahl_registrierte_Flugobjekte}")

# Anlage neues RDD um MapReduce auf die Airlines durchführen zu können
aircraftRDD = (
    aircraft_raw_rdd.filter(lambda line: len(line.split(","))>16)
    .map(lambda line: line.split(",")[17].strip('"'))
    .filter(lambda airline: airline != "''")
    .map(lambda airline: (airline, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)            
)
top_20_airlines = aircraftRDD.take(20)

print("Top 20 Airlines nach Anzahl der registrierten Flugzeuge:")
for airline, count in top_20_airlines:
    print(f"{airline}: {count}")


# Untersuchung der Datenverarbeitung
analyze_partition_distribution(aircraftRDD)

Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte 601337
Top 20 Airlines nach Anzahl der registrierten Flugzeuge:
United States Air Force: 2223
Delta Air Lines: 781
American Airlines: 657
Corporate Airlink: 649
Federal Express: 560
Royal Air Force: 503
United Airlines: 451
Southwest Airlines: 280
German Air Force: 277
Boeing: 274
United Parcel Service: 270
Indian Air Force: 236
Force Aerienne Francaise: 223
Gulfstream Aerospace: 201
Bombardier: 190
Ryanair: 159
Qatar Airways: 157
Skywest Airlines: 153
Aeroflot Russian Airlines: 147
Royal Netherlands Air Force: 144

=== Verteilung der Daten auf Partitionen ===
Anzahl der Datensätze: 4581
Datenverteilung auf Partitionen:
Partition 0: 1084 Datensätze
Partition 1: 880 Datensätze
Partition 2: 0 Datensätze
Partition 3: 2617 Datensätze


Auswertung der Bewegungsdaten mit visueller Darstellung 

In [23]:
# Analyse der Bewegungsdaten

# Einlesen der Bewegungsdaten eines definierten Zeitraums in einem bestimmten geografischen Bereich
import folium
myFileRDD = sc.textFile('data/20230126_merged.csv')
header = myFileRDD.first()

# Header entfernen
dataRDD = myFileRDD.filter(lambda line: line != header)

#Erfassen des Zeitraums in dem Daten die erhoben wurden - Skalierbarkeit gegeben aufgrund der Rückgabe eines Wertes
start_time = dataRDD.map(lambda line: line.split(',')[1]).min()
end_time = dataRDD.map(lambda line: line.split(',')[1]).max()
time_zone = start_time[-4:-1]

print(f"Die erfassten Bewegungen von Flug- und Bodenobjekten wuden im Zeitraum vom {start_time.replace('"', '').replace('+00', '')} bis zum {end_time.replace('"', '').replace('+00', '')} (Zeitzone: {time_zone}) ermittelt.")
print(f"Die Anzahl der im Zeitraum gemessenen Bewegungsdaten beträgt: {amount_of_data}")
#Geographische Analyse des betrachteten Luftraums - Skalierbarkeit gegeben aufgrund der Rückgabe und Visualisierung einzelner Werte

# Längengrade sortieren und auslesen
min_longitude = dataRDD.map(lambda line: float(line.split(',')[4])).min()
max_longitude = dataRDD.map(lambda line: float(line.split(',')[4])).max()

# Breitengrade (vermutlich Spalte mit Index 5 und 6)
min_latitude = dataRDD.map(lambda line: float(line.split(',')[3])).min()
max_latitude = dataRDD.map(lambda line: float(line.split(',')[3])).max()

# Ausgabe mit Zeitinformationen und geografischen Koordinaten
print("")
print(f"Geografische Abdeckung:")
print(f"Längengrade: von {min_longitude:.4f}° bis {max_longitude:.4f}°")
print(f"Breitengrade: von {min_latitude:.4f}° bis {max_latitude:.4f}°")

# Zentrum des Bereichs berechnen
center_lat = (min_latitude + max_latitude) / 2
center_lon = (min_longitude + max_longitude) / 2

# Karte erstellen
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

# Rechteck auf der Karte hinzufügen
bounds = [[min_latitude, min_longitude], [max_latitude, max_longitude]]
folium.Rectangle(
    bounds=bounds,
    color="red",
    weight=2,
    fill=True,
    fill_color="blue",
    fill_opacity=0.2,
    popup="Geographischer Bereich"
).add_to(m)

# Karte anzeigen
m.save("geographic_area.html")
print(f"Die Daten wurden in folgendem Luftraum erhoben:")
m



# Untersuchung der Datenverarbeitung
analyze_partition_distribution(dataRDD)

Die erfassten Bewegungen von Flug- und Bodenobjekten wuden im Zeitraum vom 2023/01/26 00:00:01 bis zum 2023/01/26 23:59:59 (Zeitzone: +00) ermittelt.
Die Anzahl der im Zeitraum gemessenen Bewegungsdaten beträgt: 1389565



Geografische Abdeckung:
Längengrade: von -12.5352° bis 13.8460°
Breitengrade: von 36.0001° bis 54.1166°
Die Daten wurden in folgendem Luftraum erhoben:

=== Verteilung der Daten auf Partitionen ===
Anzahl der Datensätze: 1389565
Datenverteilung auf Partitionen:
Partition 0: 72949 Datensätze
Partition 1: 71462 Datensätze
Partition 2: 68181 Datensätze
Partition 3: 71201 Datensätze
Partition 4: 73510 Datensätze
Partition 5: 72601 Datensätze
Partition 6: 71030 Datensätze
Partition 7: 72517 Datensätze
Partition 8: 76097 Datensätze
Partition 9: 74634 Datensätze
Partition 10: 74462 Datensätze
Partition 11: 76191 Datensätze
Partition 12: 75180 Datensätze
Partition 13: 71057 Datensätze
Partition 14: 70634 Datensätze
Partition 15: 69665 Datensätze
Partition 16: 70671 Datensätze
Partition 17: 71705 Datensätze
Partition 18: 72027 Datensätze
Partition 19: 13791 Datensätze


In [25]:
# Analyse der Bewegungsdaten nach Flug- und Bodenobjekten

# Umstrukturierung der Daten zur Analyse der Daten nach Fahrzeug
def parse_line(line):
    parts = line.split(',')
    return (
        parts[2].strip('"'),  # Key: Flugzeug-ID (icao24)
        {  # Value: Alle relevanten Daten als Dictionary
            "time": parts[1].strip('"'),
            "lat": float(parts[3].strip('"')),
            "lon": float(parts[4].strip('"')),
            "onground": int(parts[9].strip('"'))
        }
    )

# RDD transformieren: Gruppiere die Daten nach Flugzeug-ID - Skalierbarkeit gegeben aufgrund der Anlage eines neuen RDDs
flights_rdd = dataRDD.map(parse_line).groupByKey().mapValues(list)

#Anzahl der Flugzeuge, die innerhalb einer Stunde im betrachteten Bereich aktiv waren - Skalierbarkeit gegeben aufgrund der Rückgabe einzelner Werte
amount_of_aircrafts = flights_rdd.count()
print(f"Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: {amount_of_aircrafts}")




# Untersuchung der Datenverarbeitung
analyze_partition_distribution(flights_rdd)


Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: 6938

=== Verteilung der Daten auf Partitionen ===
Anzahl der Datensätze: 6938
Datenverteilung auf Partitionen:
Partition 0: 313 Datensätze
Partition 1: 356 Datensätze
Partition 2: 373 Datensätze
Partition 3: 329 Datensätze
Partition 4: 357 Datensätze
Partition 5: 336 Datensätze
Partition 6: 355 Datensätze
Partition 7: 317 Datensätze
Partition 8: 339 Datensätze
Partition 9: 306 Datensätze
Partition 10: 379 Datensätze
Partition 11: 325 Datensätze
Partition 12: 328 Datensätze
Partition 13: 354 Datensätze
Partition 14: 360 Datensätze
Partition 15: 362 Datensätze
Partition 16: 383 Datensätze
Partition 17: 337 Datensätze
Partition 18: 396 Datensätze
Partition 19: 333 Datensätze


In [26]:
# Kombination der Bewegungsdaten mit den Daten der registrierten Flug- und Bodenobjekten

def parse_type(line):
    parts = line.split(',')
    # Baseline-Verarbeitung ohne unnötige Komplexität
    return (
        parts[0].strip("'"),  # Flugzeug-ID (icao24)
        {
            "description": parts[5].strip("'") if len(parts) > 4 else "",
            "manufacturer": parts[13].strip("'") if len(parts) > 12 else "",
            "model": parts[14].strip("'") if len(parts) > 14 else "",
            "type": parts[15].strip("'") if len(parts) > 15 else "",
            "airline": parts[18].strip("'") if len(parts) > 16 else ""
        }
    )

# RDD bereinigen und nur relevante Daten aufnehmen
cleaned_type_rdd = aircraft_raw_rdd.map(parse_type)

# Kombination der Bewegunsdaten mit der Beschreibung der registrierten Flug- und Bodenobjekten
result_rdd = flights_rdd.leftOuterJoin(cleaned_type_rdd)

# Kombination der Daten in einem RDD
combined_rdd = result_rdd.map(lambda x: (
    x[0],  # Flugzeug-ID
    {
        "aircraft_info": x[1][1] if x[1][1] else {},  # Beschreibung, falls vorhanden
        "flight_data": sorted(x[1][0], key=lambda entry: entry["time"]) if x[1][0] else [] #Flugdaten sortiert nach Datum (lexografisch)
    }
))




# Untersuchung der Datenverarbeitung
analyze_partition_distribution(combined_rdd)


=== Verteilung der Daten auf Partitionen ===
Anzahl der Datensätze: 6938
Datenverteilung auf Partitionen:
Partition 0: 310 Datensätze
Partition 1: 261 Datensätze
Partition 2: 314 Datensätze
Partition 3: 272 Datensätze
Partition 4: 288 Datensätze
Partition 5: 292 Datensätze
Partition 6: 328 Datensätze
Partition 7: 277 Datensätze
Partition 8: 289 Datensätze
Partition 9: 275 Datensätze
Partition 10: 294 Datensätze
Partition 11: 266 Datensätze
Partition 12: 287 Datensätze
Partition 13: 279 Datensätze
Partition 14: 351 Datensätze
Partition 15: 276 Datensätze
Partition 16: 274 Datensätze
Partition 17: 292 Datensätze
Partition 18: 293 Datensätze
Partition 19: 290 Datensätze
Partition 20: 272 Datensätze
Partition 21: 290 Datensätze
Partition 22: 283 Datensätze
Partition 23: 285 Datensätze


In [ ]:
import csv
import math

# Haversine-Funktion zum Berechnen der Distanz zwischen zwei Punkten
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Erd-Radius in Kilometern
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2.0) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Airports.csv einlesen (Flughafendaten als RDD)
airports_path = "data/airports.csv"
airports_rdd = sc.textFile(airports_path)\
    .map(lambda line: line.strip().split(','))\
    .filter(lambda x: x[0] != "id" and len(x) > 5 and x[4].replace('.', '', 1).isdigit() and x[5].replace('.', '', 1).isdigit())\
    .map(lambda x: (x[1].strip(), x[3].strip(), float(x[4]), float(x[5])))  # (ID, Name, Latitude, Longitude)

def find_nearest_airport(lat, lon, airports):
    """
    Findet den nächsten Flughafen zu gegebenen Koordinaten innerhalb von 10 km.
    Gibt den Flughafen mit der geringsten Distanz zurück.
    """
    nearest_airport = None
    min_distance = float("inf")

    for airport in airports:
        airport_id, airport_name, airport_lat, airport_lon = airport
        distance = haversine(lat, lon, airport_lat, airport_lon)
        if distance <= 10 and distance < min_distance:  # Innerhalb von 10 km und geringste Distanz
            min_distance = distance
            nearest_airport = (airport_id, airport_name)
    
    if nearest_airport:
        return nearest_airport
    return None, None

def extract_flights_with_airports(record, airports):
    """
    Extrahiert Flüge und fügt Flughafendaten hinzu.
    """
    icao24, combined_rdd = record
    aircraft_info = combined_rdd['aircraft_info']
    flight_data = combined_rdd['flight_data']

    flights = []
    current_flight = []
    in_flight = False

    for entry in flight_data:
        if not in_flight and entry['onground'] == 0:  # Start eines Fluges
            in_flight = True
            current_flight = [entry]  # Startpunkt hinzufügen
        elif in_flight and entry['onground'] == 1:  # Landung erkannt
            current_flight.append(entry)  # Landepunkt hinzufügen
            # Flug-Informationen extrahieren
            start = current_flight[0]
            end = current_flight[-1]

            start_airport_id, start_airport_name = find_nearest_airport(start['lat'], start['lon'], airports)
            end_airport_id, end_airport_name = find_nearest_airport(end['lat'], end['lon'], airports)

            flight_info = {
                'start_time': start['time'],
                'start_coords': (start['lat'], start['lon']),
                'start_airport': {'id': start_airport_id, 'name': start_airport_name},
                'end_time': end['time'],
                'end_coords': (end['lat'], end['lon']),
                'end_airport': {'id': end_airport_id, 'name': end_airport_name},
                'route': current_flight[1:-1]  # Alle Zwischenpunkte
            }
            flights.append((icao24, aircraft_info, flight_info))
            in_flight = False
            current_flight = []
        elif in_flight:  # Punkte während des Fluges sammeln
            current_flight.append(entry)
    return flights

# Airports-Daten als Liste sammeln (Broadcast-Variable verwenden)
airports_list = airports_rdd.collect()
broadcast_airports = sc.broadcast(airports_list)

# Anwenden der Extraktionsfunktion auf die RDD mit Flughafendaten
flights_rdd = combined_rdd.flatMap(lambda record: extract_flights_with_airports(record, broadcast_airports.value))

#lfpo_flights_count = flights_rdd.filter(lambda flight: flight[2]['end_airport']['id'] == 'LFPO').count()
#print(f"Anzahl der Flüge mit end_airport ID 'LFPO': {lfpo_flights_count}")
flights_rdd.take(1)



# Untersuchung der Datenverarbeitung
analyze_partition_distribution(flights_rdd)

In [ ]:
conf.set("spark.network.timeout", "600s")
conf.set("spark.executor.heartbeatInterval", "120s")
conf.set("spark.shuffle.io.retryWait", "5s")
conf.set("spark.shuffle.io.maxRetries", "10")

import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.graph_objects as go

# Beispiel-Daten erweitern: Liste von Flügen
flights_data = flights_rdd.collect()

# Schritt 1: Flugverbindungen aggregieren (Start- und End-Airport-Paare zählen)
connections = defaultdict(int)
start_end_names = {}

for _, _, flight_info in flights_data:
    start_airport = flight_info['start_airport']
    end_airport = flight_info['end_airport']
    
    if start_airport['id'] and end_airport['id']:  # Nur valide Airports berücksichtigen
        start_id, start_name = start_airport['id'], start_airport['name']
        end_id, end_name = end_airport['id'], end_airport['name']
        
        connections[(start_id, end_id)] += 1
        start_end_names[start_id] = start_name
        start_end_names[end_id] = end_name

# Schritt 2: Netzwerkdiagramm erstellen
G = nx.Graph()

# Knoten und Kanten hinzufügen (nur Verbindungen mit mehr als 5 Flügen)
for (start, end), weight in connections.items():
    if weight > 5:  # Filter: Nur Verbindungen > 5
        G.add_edge(start, end, weight=weight)

# Schritt 3: Positionen berechnen
pos = nx.spring_layout(G, seed=42)

# Knoten und Kanten für Plotly vorbereiten
edges_x = []
edges_y = []
weights = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    weights.append(edge[2]['weight'])

# Knoten vorbereiten
nodes_x = []
nodes_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    nodes_x.append(x)
    nodes_y.append(y)
    node_text.append(start_end_names.get(node, node))  # Namen verwenden

# Schritt 4: Interaktives Diagramm mit Plotly erstellen
fig = go.Figure()

# Kanten hinzufügen mit stärkerer Gewichtung
max_weight = max(weights) if weights else 1  # Maximale Gewichtung für Skalierung

for i, edge in enumerate(G.edges(data=True)):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=(weights[i] / max_weight) * 100, color='black'),  # Stärkere Skalierung (100-fach)
        hoverinfo='none',
        mode='lines'))

# Knoten hinzufügen
fig.add_trace(go.Scatter(
    x=nodes_x, y=nodes_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    marker=dict(
        size=10,
        color='skyblue',
        line_width=2),
    hoverinfo='text'))

# Layout anpassen
fig.update_layout(
    title="Interaktives Netzwerkdiagramm der Flugverbindungen (Verbindungen > 5)",
    title_x=0.5,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=30),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white'
)

# Interaktives Diagramm anzeigen
fig.show()


In [27]:
#Analyse des kombinierten RDDs (Bewegungsdaten + Beschreibung)

#Ermittlung der Anzahl an Flug-Datensätze pro Flugzeug
countdatasetsRDD = combined_rdd.map(lambda x: (x[0],len(x[1]['flight_data']))).sortBy(lambda x: x[1], ascending=False)
first_10_elements = countdatasetsRDD.take(10)
print(f"Flugzeuge mit den meisten Datensätzen (Top 10) sind: {first_10_elements}  ")

print("")

# Bestimmung der Positon - Skalierbarkeit gegeben über Betrachtung einzelner Werte
airRDD = combined_rdd.map(lambda x: (x[0], x[1]['flight_data'][-1]["onground"]))
onground = airRDD.filter(lambda x: x[1]==1).count()
inair = airRDD.filter(lambda x: x[1]==0).count()
print(f"Status der Objekt zum letzten gemessenen Zeitpunkt ({end_time.replace('"', '').replace('+00', '')}): ")
print("Objekte aktuell in der Luft: " + str(onground))
print("Objekte aktuell am Boden: " + str(inair))


# Ermittlung der Position im Zeitraum - Skalierbarkeit gegeben über Betrachtung einzelner Werte
nextRDD = combined_rdd.map(lambda x: (x[0], sum(datapoint["onground"] for datapoint in x[1]['flight_data']), len(x[1]['flight_data'])))
calculatedRDD = nextRDD.map(lambda x: (x[0], x[1]/x[2]))

not_started = calculatedRDD.filter(lambda x: x[1] == 1).count()
print("")
print(f"Im Zeitraum: {start_time.replace('"', '').replace('+00', '')} - {end_time.replace('"', '').replace('+00', '')} (Zeitzone: {time_zone}) sind: ")
print(f"Objekte nicht am Boden geblieben: {not_started}")
not_landed = calculatedRDD.filter(lambda x: x[1] == 0).count()
print(f"Objekte nicht gelandet: {not_landed}")
startet_or_landed = calculatedRDD.filter(lambda x: x[1] < 1 and x[1] >0).count()
print(f"Objekte gestartet oder gelandet: {startet_or_landed}")


# Anzahl Starts und Anzahl Landungen

Flugzeuge mit den meisten Datensätzen (Top 10) sind: [('392adb', 1440), ('490031', 1439), ('425851', 1439), ('425850', 1437), ('344043', 1437), ('425855', 1437), ('490020', 1437), ('392afb', 1435), ('425852', 1435), ('425854', 1427)]  

Status der Objekt zum letzten gemessenen Zeitpunkt (2023/01/26 23:59:59): 
Objekte aktuell in der Luft: 2277
Objekte aktuell am Boden: 4661

Im Zeitraum: 2023/01/26 00:00:01 - 2023/01/26 23:59:59 (Zeitzone: +00) sind: 
Objekte nicht am Boden geblieben: 603
Objekte nicht gelandet: 2201
Objekte gestartet oder gelandet: 4134


In [ ]:
# Visualisierung der Bewegung und Beschreibung eines Flug- oder Bodenobjektes - Skalierbarkeit gegeben über die Betrachtung eines Objektes

import folium
from IPython.display import display

# Die Flugzeug-ID, die visualisiert werden soll
selected_icao24 = "3964e4"

# Filtere die Daten für die spezifische Flugzeug-ID im RDD
filtered_data = combined_rdd.filter(lambda x: x[0] == selected_icao24).collect()

if filtered_data:
    # Hole die Daten für das ausgewählte Flugzeug
    icao24, route_data = filtered_data[0]
    aircraft_info = route_data["aircraft_info"]  # Zugriff auf die Flugzeugbeschreibung
    flight_data = route_data["flight_data"]  # Zugriff auf die Flugdaten

    print(f"Flugzeug-ID: {icao24}")

    # Erstelle die Karte, zentriert auf den ersten Datenpunkt
    first_lat = flight_data[0]["lat"]
    first_lon = flight_data[0]["lon"]
    mymap = folium.Map(location=[first_lat, first_lon], zoom_start=12)

    # Zeichne Linien zwischen aufeinanderfolgenden Punkten
    for i in range(len(flight_data) - 1):
        point_a = flight_data[i]
        point_b = flight_data[i + 1]

        # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
        if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
            folium.PolyLine(
                locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                color="blue",
                weight=2.5,
                opacity=0.8
            ).add_to(mymap)

    # Füge Marker nur für Landepunkte hinzu
    for point in flight_data:
        if point["onground"] == 1:  # Nur Landepunkte anzeigen
            folium.Marker(
                location=(point["lat"], point["lon"]),
                popup=f"Zeit: {point['time']}<br>Am Boden: {point['onground']}<br>Hersteller: {aircraft_info['manufacturer']}<br>Flugzeugtyp: {aircraft_info['model']}<br>Airline: {aircraft_info['airline']}",
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(mymap)

    # Zeige die Karte im Notebook
    display(mymap)
else:
    print(f"Keine Daten für Flugzeug-ID '{selected_icao24}' gefunden.")


In [ ]:
# Visualisierung der Bewegung und Beschreibung von mehreren Flug- oder Bodenobjekten - Skalierbarkeit gegeben über die Betrachtung einzelner Objektes

import folium
from IPython.display import display
import random

# Zufällige Farben für die Flugzeuge
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Nimm eine beliebige Anzahl zufälliger Flugzeuge aus dem RDD
random_aircrafts = combined_rdd.takeSample(withReplacement=False, num=10)

# Erstelle eine Karte, zentriert auf einen Beispielpunkt
if random_aircrafts:
    # Hole den ersten Punkt zur Zentrierung der Karte
    first_lat = random_aircrafts[0][1]["flight_data"][0]["lat"]
    first_lon = random_aircrafts[0][1]["flight_data"][0]["lon"]
    mymap = folium.Map(location=[first_lat, first_lon], zoom_start=6)

    # Iteriere über die zufälligen Flugzeuge
    for icao24, data in random_aircrafts:
        aircraft_info = data["aircraft_info"]  # Infos über das Flugzeug
        flight_data = data["flight_data"]      # Flugdaten

        # Sichere Abfrage von Typ, Modell und Hersteller mit Standardwerten
        aircraft_type = aircraft_info.get("type", "Unknown")
        aircraft_model = aircraft_info.get("model", "Unknown")
        aircraft_manufacturer = aircraft_info.get("manufacturer", "Unknown")
        aircraft_airline = aircraft_info.get("airline", "Unknown")

        # Generiere eine zufällige Farbe für die Linien
        color = get_random_color()

        # Zeichne Linien zwischen aufeinanderfolgenden Punkten
        for i in range(len(flight_data) - 1):
            point_a = flight_data[i]
            point_b = flight_data[i + 1]

            # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
            if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                folium.PolyLine(
                    locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                    color=color,
                    weight=2.5,
                    opacity=0.8
                ).add_to(mymap)

        # Füge Marker nur für Landepunkte hinzu
        for point in flight_data:
            if point["onground"] == 1:  # Nur Landepunkte anzeigen
                folium.Marker(
                    location=(point["lat"], point["lon"]),
                    popup=(
                        f"Flugzeug: {icao24}<br>"
                        f"Hersteller: {aircraft_manufacturer}<br>"
                        f"Modell: {aircraft_model}<br>"
                        f"Airline: {aircraft_airline}<br>"
                        f"Zeit: {point['time']}<br>"
                        f"Am Boden: {point['onground']}"
                    ),
                    icon=folium.Icon(color="green", icon="info-sign")
                ).add_to(mymap)

    # Zeige die Karte im Notebook
    display(mymap)
else:
    print("Keine Daten gefunden.")


In [ ]:
# Map und Filter: Direktes Überprüfen der Konvertierbarkeit in float
# 1 Dezimalstelle: ~11,1 km Genauigkeit eine Nachkommastelle reicht für globale Karten oder grobe Cluster.
# 2 Dezimalstellen: ~1,1 km Genauigkeit gut für Städte oder Regionen.
# 3 Dezimalstellen: ~110 m Genauigkeit ideal für Stadtviertel oder grobe Stadtanalysen.
# Ab 3 Nachkommastellen geht nix mehr
def extract_coordinates(line):
    try:
        parts = line.split(',')
        return round(float(parts[3]), 2), round(float(parts[4]), 2)  # Breitengrad, Längengrad
    except (ValueError, IndexError):
        return None  # Falls Konvertierung oder Zugriff fehlschlägt

location_rdd = dataRDD.map(extract_coordinates).filter(lambda x: x is not None)

# Ergebnisse anzeigen
print(location_rdd.take(5))


number = location_rdd.count()
print(f"Anzahld er Daten vor der Reduktion {number}")


# Map-Reduce: Zähle, wie oft jede Kombination von Breitengrad und Längengrad vorkommt
coordinate_counts = location_rdd.map(lambda coord: (coord, 1)).reduceByKey(lambda a, b: a + b)

# Ergebnisse anzeigen
print(coordinate_counts.take(5))
print(f"Anzahl Koordinaten nach der Reduktion {coordinate_counts.count()}")

In [ ]:
import folium
from folium.plugins import HeatMap

# Beispiel: Aggregierte Koordinaten mit Häufigkeiten (ersetzt durch deine Daten)
aggregated_data = coordinate_counts.collect()

# Vorbereitung der Heatmap-Daten
heatmap_data = [(lat_lon[0], lat_lon[1], count) for lat_lon, count in aggregated_data]

# Erstelle die Karte
if heatmap_data:
    # Zentriere die Karte auf den ersten Punkt und passe die Zoom-Stufe an
    mymap = folium.Map(location=[heatmap_data[0][0], heatmap_data[0][1]], zoom_start=6)

    # HeatMap hinzufügen (Radius und max_zoom können angepasst werden)
    HeatMap(heatmap_data, radius=15, max_zoom=13).add_to(mymap)

    # Karte anzeigen
    from IPython.display import display
    display(mymap)
else:
    print("Keine gültigen Punkte für die Heatmap gefunden.")